In [ ]:
from app.repositories import CardRepository
import mlx_whisper
from typing import List, Dict, Any, Optional
import os

DEFAULT_MODEL = "mlx-community/whisper-small-mlx"

def carregar_modelo(modelo: str = DEFAULT_MODEL) -> Any:
    """
    Carrega o modelo de transcrição de áudio.

    Args:
        modelo: Caminho ou repositório HuggingFace do modelo. Por padrão usa o modelo whisper-small-mlx.

    Returns:
        Modelo carregado
    """
    print(f"Carregando modelo {modelo}...")
    return modelo  # O MLX Whisper não precisa carregar o modelo explicitamente antes de transcrever

def transcrever_audio(caminho_audio: str, modelo: str = DEFAULT_MODEL, timestamps_palavras: bool = True) -> Dict[str, Any]:
    """
    Transcreve um arquivo de áudio para texto.

    Args:
        caminho_audio: Caminho para o arquivo de áudio
        modelo: Modelo a ser usado para transcrição
        timestamps_palavras: Se deve incluir timestamps para cada palavra

    Returns:
        Dicionário com a transcrição e metadados
    """
    if not os.path.exists(caminho_audio):
        raise FileNotFoundError(f"Arquivo de áudio não encontrado: {caminho_audio}")

    print(f"Transcrevendo áudio: {caminho_audio}")
    resultado = mlx_whisper.transcribe(
        caminho_audio,
        path_or_hf_repo=modelo,
        word_timestamps=timestamps_palavras,
    )

    return resultado

def extrair_palavras(resultado_transcricao: Dict[str, Any]) -> List[Dict[str, Any]]:
    """
    Extrai todas as palavras individuais da transcrição com seus timestamps.

    Args:
        resultado_transcricao: Resultado da transcrição

    Returns:
        Lista de dicionários, cada um contendo uma palavra e seus metadados
    """
    palavras = []

    for segmento in resultado_transcricao["segments"]:
        if "words" in segmento:
            for palavra in segmento["words"]:
                palavras.append({
                    "palavra": palavra["word"],
                    "inicio": palavra.get("start", 0),
                    "fim": palavra.get("end", 0),
                    "probabilidade": palavra.get("probability", 1.0)
                })

    return palavras

def salvar_transcricao(palavras: List[Dict[str, Any]], repositorio: Optional[CardRepository] = None) -> None:
    """
    Salva as palavras transcritas no repositório.

    Args:
        palavras: Lista de palavras transcritas
        repositorio: Repositório para salvar os dados (opcional)
    """
    if repositorio is None:
        print("Nenhum repositório fornecido, apenas imprimindo as palavras:")
        for palavra in palavras:
            print(palavra)
        return

    # Lógica para salvar no repositório
    for palavra in palavras:
        # Implementação depende da interface do CardRepository
        pass

def converter_audio_para_texto(caminho_audio: str, modelo: str = DEFAULT_MODEL) -> List[Dict[str, Any]]:
    """
    Função principal que executa o processo completo de conversão.

    Args:
        caminho_audio: Caminho para o arquivo de áudio
        modelo: Modelo a ser usado para transcrição

    Returns:
        Lista de palavras transcritas
    """
    modelo_carregado = carregar_modelo(modelo)
    resultado = transcrever_audio(caminho_audio, modelo_carregado)
    palavras = extrair_palavras(resultado)
    return palavras

if __name__ == "__main__":
    print("Iniciando conversão de áudio para texto...")

    caminho_audio = "meu_audio.webm"
    resultado = transcrever_audio(caminho_audio, DEFAULT_MODEL)

    palavras = extrair_palavras(resultado)
    salvar_transcricao(palavras)

    print(f"Transcrição concluída! {len(palavras)} palavras encontradas.")